In [ ]:
from multiinstance.data.gaussian_dg import GaussianMixtureDataGenerator

import numpy as np

from easydict import EasyDict
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Lambda, Activation, Concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as KB
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow import summary
import tensorflow.keras.backend as K
import tensorflow as tf

In [ ]:
def generateParams(nClusters, nComps, dim,aucpn_range=[0.8,0.85]):
    posMeans,posCovs,negMeans,negCovs,dgs =[], [],[],[],[]
    for comp in range(nClusters):
        dg = GaussianMixtureDataGenerator(dim,nComps,aucpn_range,100,1000)
        pos,neg = dg.components_pos[0], dg.components_neg[0]
        r = np.zeros(dim)
        for _ in range(1):
            r += np.random.normal(loc=0,scale=10,size=dim)
        posMeans.append(pos.mean+r)
        posCovs.append(pos.cov)
        negMeans.append(neg.mean+r)
        negCovs.append(neg.cov)
        dgs.append(dg)
    val = EasyDict
    val.posMeans = posMeans
    val.posCovs = posCovs
    val.negMeans = negMeans
    val.negCovs = negCovs
    val.dgs = dgs
    return val

In [ ]:
args = EasyDict()
args.nClusters = 1
args.nComps = 1
args.dim = 10
args.aucpn_range=[0.7,1.0]
args.hdim = 1000
args.latent_dim = 2

In [ ]:
data = generateParams(args.nClusters, args.nComps, args.dim, args.aucpn_range)

In [ ]:
# x,y = data.dgs[0].pn_data(20000,.5)
# y = y.astype(bool)

x_pu, y_pu, y_pn = data.dgs[0].pu_data()

In [ ]:
encoder = Sequential(name="encoder")
encoder.add(Dense(args.hdim, input_shape=(args.dim,), activation='relu',))
encoder.add(Dense(args.hdim, activation='relu'))
encoder.add(Dense(args.latent_dim, activation=None))

decoder = Sequential(layers=[Dense(args.hdim, input_shape=(args.latent_dim,),activation="relu"),
                             Dense(args.hdim,activation="relu"),
                             Dense(args.dim,activation="linear")],name="decoder")

discriminator = Sequential(layers=[Dense(args.hdim,input_shape=(args.latent_dim,),activation="relu"),
                                   Dense(args.hdim, activation="relu"),
                                   Dense(1, activation="sigmoid")],name="discriminator")

autoencoder = Sequential(layers=[encoder, decoder])
model_enc_disc = Sequential(layers=[encoder, discriminator])